In [1]:
from road_network.cython_cars_and_roads import VoidGenerator, Line, LineWLight, RoadManager, Crossroad

In [1]:
from road_network.road_and_cars import VoidGenerator, Line, LineWLight, Crossroad, CarManager, RoadManager

In [2]:
lineW0 = LineWLight(20, 18, 20, 10)
lineW1 = LineWLight(20, 18, 20, 10)
lineW2 = LineWLight(20, 18, 20, 10, 15)
lineW3 = LineWLight(20, 18, 20, 10, 15)

line0 = Line(20)
line1 = Line(20)
line2 = Line(20)
line3 = Line(20)

crossroad = Crossroad(1, 1, 1, 1)

void = VoidGenerator(.6)

In [3]:
lineW0.add_output(crossroad, 0, 0)
lineW1.add_output(crossroad, 0, 1)
lineW2.add_output(crossroad, 0, 2)
lineW3.add_output(crossroad, 0, 3)

crossroad.add_output(line0, 0)
crossroad.add_output(line1, 1)
crossroad.add_output(line2, 2)
crossroad.add_output(line3, 3)

void.add_output(lineW0, 0)
void.add_output(lineW1, 1)
void.add_output(lineW2, 2)
void.add_output(lineW3, 3)

line0.add_output(void, 0, 0)
line1.add_output(void, 0, 1)
line2.add_output(void, 0, 2)
line3.add_output(void, 0, 3)

In [4]:
%%timeit
for frame in range(400):
    for road in RoadManager.roads:
        road.move_cars(frame)
    
    for road in RoadManager.roads:
        road.step(frame)
    
    for road in RoadManager.roads:
        road.process_outputs()

2.22 s ± 321 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
for road in RoadManager.roads:
        road.move_cars(frame)

3.14 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
for road in RoadManager.roads:
        road.step(frame)

538 µs ± 129 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
for road in RoadManager.roads:
        road.process_outputs()

926 µs ± 158 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
dfghjhgf

In [11]:
%%timeit
for road in RoadManager.roads:
        road.move_cars(frame)

2.93 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
for road in RoadManager.roads:
        road.step(frame)

1.25 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
for road in RoadManager.roads:
        road.process_outputs()

852 µs ± 252 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
